In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
import glob

In [2]:
Phil = pd.read_csv('cleaned_files.csv', low_memory=False)

**We noticed while visualizing that there are more bike trips than unique bike trip.**

In [3]:
Phil['trip_id'].nunique()

1679150

In [4]:
Phil['trip_id'].count()

1781474

In [5]:
# 1781474 - 1679150 = 102324 
# so there must be 102324 * 2 = 204648 duplicates
duplicates = Phil[Phil.duplicated(subset='trip_id', keep=False)]
duplicates

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
496588,418016541,9,2021-09-01 00:00:00,2021-09-01 00:09:00,3010,3248,18371,30,One Way,Indego30,electric,2021-09-01,2,0,2021-09-01,2,0,"39.947109,-75.166183","39.942131,-75.18335"
496589,418016540,10,2021-09-01 00:05:00,2021-09-01 00:15:00,3158,3190,19503,30,One Way,Indego30,electric,2021-09-01,2,0,2021-09-01,2,0,"39.925522,-75.169037","39.948921,-75.169907"
496590,418016539,3,2021-09-01 00:09:00,2021-09-01 00:12:00,3009,3022,14723,30,One Way,Indego30,standard,2021-09-01,2,0,2021-09-01,2,0,"39.955761,-75.189819","39.95472,-75.183228"
496591,418016538,12,2021-09-01 00:09:00,2021-09-01 00:21:00,3062,3053,2480,30,One Way,Indego30,standard,2021-09-01,2,0,2021-09-01,2,0,"39.951969,-75.179428","39.932308,-75.181541"
496592,418016537,9,2021-09-01 00:09:00,2021-09-01 00:18:00,3047,3040,16950,30,One Way,Indego30,electric,2021-09-01,2,0,2021-09-01,2,0,"39.95071,-75.149208","39.962891,-75.166061"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701231,428365176,10,2021-09-30 23:57:00,2021-10-01 00:07:00,3009,3035,18791,30,One Way,Indego30,electric,2021-09-30,3,23,2021-10-01,4,0,"39.955761,-75.189819","39.962711,-75.194191"
701232,428365174,7,2021-09-30 23:57:00,2021-10-01 00:04:00,3047,3028,5263,30,One Way,Indego30,standard,2021-09-30,3,23,2021-10-01,4,0,"39.95071,-75.149208","39.940609,-75.149582"
701233,428365172,7,2021-09-30 23:58:00,2021-10-01 00:05:00,3046,3050,18675,30,One Way,Indego30,electric,2021-09-30,3,23,2021-10-01,4,0,"39.950119,-75.144722","39.953388,-75.154259"
701234,428365170,3,2021-09-30 23:58:00,2021-10-01 00:01:00,3115,3075,21618,30,One Way,Indego30,electric,2021-09-30,3,23,2021-10-01,4,0,"39.97263,-75.167572","39.967178,-75.161247"


**Let's see if the duplicates are also always only two pairs**

In [6]:
subset_cols = ['trip_id', 'bike_id', 'start_time']

# Group the duplicates by the subset of columns and count the number of occurrences
duplicate_counts = duplicates.groupby(subset_cols).size().reset_index(name='duplicate_count')
duplicate_counts

,trip_id,bike_id,start_time,duplicate_count
0,418016501,2537,2021-09-01 01:48:00,2
1,418016502,2697,2021-09-01 01:39:00,2
2,418016503,14567,2021-09-01 01:34:00,2
3,418016504,19971,2021-09-01 01:30:00,2
4,418016505,16748,2021-09-01 01:30:00,2
...,...,...,...,...
102319,428365209,14638,2021-09-30 23:42:00,2
102320,428365227,19091,2021-09-30 23:36:00,2
102321,428365230,18745,2021-09-30 23:35:00,2
102322,428365316,11887,2021-09-30 23:08:00,2


In [7]:
duplicate_counts['duplicate_count'].unique()

array([2], dtype=int64)

**As we can see there are only two pairs**

In [8]:
trip_ids = [418016501, 418016503, 418016505, 428365209, 428365230, 428647877]
dupli = Phil[Phil['trip_id'].isin(trip_ids)]
dupli

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
496622,418016505,10,2021-09-01 01:30:00,2021-09-01 01:40:00,3201,3104,16748,30,One Way,Indego30,electric,2021-09-01,2,1,2021-09-01,2,1,"39.955231,-75.166199","39.96664,-75.192093"
496624,418016503,20,2021-09-01 01:34:00,2021-09-01 01:54:00,3168,3251,14567,1,One Way,Day Pass,standard,2021-09-01,2,1,2021-09-01,2,1,"39.95134,-75.173943","39.933659,-75.193192"
496626,418016501,7,2021-09-01 01:48:00,2021-09-01 01:55:00,3007,3154,2537,30,One Way,Indego30,standard,2021-09-01,2,1,2021-09-01,2,1,"39.945171,-75.159927","39.95924,-75.158211"
583710,428647877,1440,2021-09-26 17:45:00,2021-10-01 04:04:00,3058,3210,19561,30,One Way,Indego30,electric,2021-09-26,6,17,2021-10-01,4,4,"39.967159,-75.170013","39.984921,-75.156677"
598885,428365230,29,2021-09-30 23:35:00,2021-10-01 00:04:00,3125,3009,18745,1,One Way,Day Pass,electric,2021-09-30,3,23,2021-10-01,4,0,"39.943909,-75.167351","39.955761,-75.189819"
598894,428365209,19,2021-09-30 23:42:00,2021-10-01 00:01:00,3202,3100,14638,30,One Way,Indego30,standard,2021-09-30,3,23,2021-10-01,4,0,"39.95134,-75.16758","39.927769,-75.151031"
598946,418016505,10,2021-09-01 01:30:00,2021-09-01 01:40:00,3201,3104,16748,30,One Way,Indego30,electric,2021-09-01,2,1,2021-09-01,2,1,"39.955231,-75.166199","39.96664,-75.192093"
598948,418016503,20,2021-09-01 01:34:00,2021-09-01 01:54:00,3168,3251,14567,1,One Way,Day Pass,standard,2021-09-01,2,1,2021-09-01,2,1,"39.95134,-75.173943","39.933659,-75.193192"
598950,418016501,7,2021-09-01 01:48:00,2021-09-01 01:55:00,3007,3154,2537,30,One Way,Indego30,standard,2021-09-01,2,1,2021-09-01,2,1,"39.945171,-75.159927","39.95924,-75.158211"
686034,428647877,1440,2021-09-26 17:45:00,2021-10-01 04:04:00,3058,3210,19561,30,One Way,Indego30,electric,2021-09-26,6,17,2021-10-01,4,4,"39.967159,-75.170013","39.984921,-75.156677"


In [9]:
# trip_id = 428647877,  duration = 1440min and starttime and endtime disbalanced maybe there are more
# requires further investigation

In [10]:
duplii = Phil[Phil['trip_id'] == 428647877]
duplii

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
583710,428647877,1440,2021-09-26 17:45:00,2021-10-01 04:04:00,3058,3210,19561,30,One Way,Indego30,electric,2021-09-26,6,17,2021-10-01,4,4,"39.967159,-75.170013","39.984921,-75.156677"
686034,428647877,1440,2021-09-26 17:45:00,2021-10-01 04:04:00,3058,3210,19561,30,One Way,Indego30,electric,2021-09-26,6,17,2021-10-01,4,4,"39.967159,-75.170013","39.984921,-75.156677"


In [11]:
Phil = Phil.drop_duplicates(subset='trip_id', keep='first')
Phil

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
0,365184634,15,2021-01-01 00:05:00,2021-01-01 00:20:00,3125,3157,14542,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.943909,-75.167351","39.925449,-75.159538"
1,365184633,14,2021-01-01 00:06:00,2021-01-01 00:20:00,3022,3010,5294,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.95472,-75.183228","39.947109,-75.166183"
2,365184632,33,2021-01-01 00:07:00,2021-01-01 00:40:00,3066,3201,14586,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.94561,-75.173477","39.955231,-75.166199"
3,365184631,9,2021-01-01 00:13:00,2021-01-01 00:22:00,3045,3034,14683,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947922,-75.162369","39.933151,-75.162483"
4,365184630,13,2021-01-01 00:37:00,2021-01-01 00:50:00,3052,3040,3330,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947319,-75.156952","39.962891,-75.166061"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781469,579499101,23,2022-12-31 23:44:00,2023-01-01 00:07:00,3244,3287,14674,30,One Way,Indego30,standard,2022-12-31,5,23,2023-01-01,6,0,"39.938648,-75.16674","39.943668,-75.216057"
1781470,579499099,17,2022-12-31 23:45:00,2023-01-01 00:02:00,3295,3054,23113,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950279,-75.160271","39.962502,-75.174202"
1781471,579499097,8,2022-12-31 23:51:00,2022-12-31 23:59:00,3255,3046,16897,30,One Way,Indego30,electric,2022-12-31,5,23,2022-12-31,5,23,"39.950951,-75.164383","39.950119,-75.144722"
1781472,579499095,23,2022-12-31 23:54:00,2023-01-01 00:16:00,3255,3113,22646,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950951,-75.164383","39.97472,-75.197807"


**Now we want to see if the endtime - starttime also results in the given duration. and also delete the 1440 min durations if they really correspond to the calculated endtime - starttime**

In [12]:
mask = Phil['duration'] == 1440
mask1440 = Phil[mask]
mask1440

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
3508,365545144,1440,2021-01-05 16:49:00,2021-01-07 00:08:00,3055,3041,19793,30,One Way,Indego30,electric,2021-01-05,1,16,2021-01-07,3,0,"39.951118,-75.154572","39.968491,-75.13546"
32056,367341249,1440,2021-02-06 21:38:00,2021-02-07 21:45:00,3021,3021,5303,1,Round Trip,Day Pass,standard,2021-02-06,5,21,2021-02-07,6,21,"39.953899,-75.169022","39.953899,-75.169022"
34793,367725064,1440,2021-02-11 09:18:00,2021-02-15 08:57:00,3213,3213,3280,365,Round Trip,Indego365,standard,2021-02-11,3,9,2021-02-15,0,8,"39.938869,-75.166634","39.938869,-75.166634"
36738,367722628,1440,2021-02-14 15:45:00,2021-02-15 16:10:00,3154,3154,3612,30,Round Trip,Indego30,standard,2021-02-14,6,15,2021-02-15,0,16,"39.95924,-75.158211","39.95924,-75.158211"
37195,367828978,1440,2021-02-15 11:08:00,2021-02-17 08:49:00,3073,3073,3703,30,Round Trip,Indego30,standard,2021-02-15,0,11,2021-02-17,2,8,"39.96143,-75.15242","39.96143,-75.15242"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774669,578068516,1440,2022-12-23 22:49:00,2022-12-25 04:25:00,3210,3210,16962,30,Round Trip,Indego30,electric,2022-12-23,4,22,2022-12-25,6,4,"39.984921,-75.156677","39.984921,-75.156677"
1779800,579793526,1440,2022-12-30 14:19:00,2023-01-02 04:25:00,3212,3212,23229,1,Round Trip,Day Pass,electric,2022-12-30,4,14,2023-01-02,0,4,"39.963791,-75.18158","39.963791,-75.18158"
1780277,579859025,1440,2022-12-30 17:26:00,2023-01-02 10:42:00,3071,3071,11765,30,Round Trip,Indego30,standard,2022-12-30,4,17,2023-01-02,0,10,"39.961731,-75.187881","39.961731,-75.187881"
1781193,579647620,1440,2022-12-31 15:15:00,2023-01-01 15:46:00,3004,3057,11948,1,One Way,Day Pass,standard,2022-12-31,5,15,2023-01-01,6,15,"39.953781,-75.163742","39.96439,-75.179871"


In [13]:
Phil['start_time'] = pd.to_datetime(Phil['start_time'])
Phil['end_time'] = pd.to_datetime(Phil['end_time'])

# need to convert duration into the same format of calculated_duration
Phil['givenDuration'] = Phil['duration'].apply(lambda x: timedelta(minutes=x))
calculated_duration = Phil['end_time'] - Phil['start_time']
mask = calculated_duration == Phil['givenDuration']
mask_correctDuration = Phil[mask]
mask_correctDuration

# so these are correct ones

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
0,365184634,15,2021-01-01 00:05:00,2021-01-01 00:20:00,3125,3157,14542,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.943909,-75.167351","39.925449,-75.159538",0 days 00:15:00
1,365184633,14,2021-01-01 00:06:00,2021-01-01 00:20:00,3022,3010,5294,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.95472,-75.183228","39.947109,-75.166183",0 days 00:14:00
2,365184632,33,2021-01-01 00:07:00,2021-01-01 00:40:00,3066,3201,14586,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.94561,-75.173477","39.955231,-75.166199",0 days 00:33:00
3,365184631,9,2021-01-01 00:13:00,2021-01-01 00:22:00,3045,3034,14683,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947922,-75.162369","39.933151,-75.162483",0 days 00:09:00
4,365184630,13,2021-01-01 00:37:00,2021-01-01 00:50:00,3052,3040,3330,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947319,-75.156952","39.962891,-75.166061",0 days 00:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781468,579499103,3,2022-12-31 23:39:00,2022-12-31 23:42:00,3039,3077,4813,30,One Way,Indego30,standard,2022-12-31,5,23,2022-12-31,5,23,"39.97121,-75.159698","39.972069,-75.163513",0 days 00:03:00
1781469,579499101,23,2022-12-31 23:44:00,2023-01-01 00:07:00,3244,3287,14674,30,One Way,Indego30,standard,2022-12-31,5,23,2023-01-01,6,0,"39.938648,-75.16674","39.943668,-75.216057",0 days 00:23:00
1781470,579499099,17,2022-12-31 23:45:00,2023-01-01 00:02:00,3295,3054,23113,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950279,-75.160271","39.962502,-75.174202",0 days 00:17:00
1781471,579499097,8,2022-12-31 23:51:00,2022-12-31 23:59:00,3255,3046,16897,30,One Way,Indego30,electric,2022-12-31,5,23,2022-12-31,5,23,"39.950951,-75.164383","39.950119,-75.144722",0 days 00:08:00


In [14]:
reverse_mask = ~ mask

mask_incorrectDuration = Phil[reverse_mask]

mask_incorrectDuration

# these are the incorrect ones

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
171,365200960,59,2021-01-01 12:24:00,2021-01-01 13:22:00,3201,3101,14596,30,One Way,Indego30,standard,2021-01-01,4,12,2021-01-01,4,13,"39.955231,-75.166199","39.942951,-75.159554",0 days 00:59:00
215,365201647,45,2021-01-01 13:15:00,2021-01-01 13:59:00,3212,3212,14630,30,Round Trip,Indego30,standard,2021-01-01,4,13,2021-01-01,4,13,"39.963791,-75.18158","39.963791,-75.18158",0 days 00:45:00
222,365201631,26,2021-01-01 13:34:00,2021-01-01 13:59:00,3166,3182,14519,365,One Way,Indego365,standard,2021-01-01,4,13,2021-01-01,4,13,"39.972351,-75.134453","39.950809,-75.169533",0 days 00:26:00
384,365247227,6,2021-01-02 06:18:00,2021-01-02 06:23:00,3169,3033,17642,365,One Way,Indego365,electric,2021-01-02,5,6,2021-01-02,5,6,"39.953819,-75.142632","39.95005,-75.156723",0 days 00:06:00
415,365249927,11,2021-01-02 08:19:00,2021-01-02 08:29:00,3121,3037,5376,365,One Way,Indego365,standard,2021-01-02,5,8,2021-01-02,5,8,"39.97414,-75.180222","39.954239,-75.161377",0 days 00:11:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781343,579439049,8,2022-12-31 17:50:00,2022-12-31 17:59:00,3010,3010,25102,30,Round Trip,Indego30,electric,2022-12-31,5,17,2022-12-31,5,17,"39.947109,-75.166183","39.947109,-75.166183",0 days 00:08:00
1781351,579447376,4,2022-12-31 18:02:00,2022-12-31 18:07:00,3236,3116,22415,365,One Way,Indego365,electric,2022-12-31,5,18,2022-12-31,5,18,"39.96566,-75.17765","39.96006,-75.171982",0 days 00:04:00
1781367,579447359,8,2022-12-31 18:29:00,2022-12-31 18:36:00,3101,3010,11004,365,One Way,Indego365,standard,2022-12-31,5,18,2022-12-31,5,18,"39.942951,-75.159554","39.947109,-75.166183",0 days 00:08:00
1781396,579482381,6,2022-12-31 19:13:00,2022-12-31 19:20:00,3100,3025,22704,30,One Way,Indego30,electric,2022-12-31,5,19,2022-12-31,5,19,"39.927769,-75.151031","39.937241,-75.161201",0 days 00:06:00


**Now let's see by how many minutes the durations are wrong**

In [15]:
mask = calculated_duration == Phil['givenDuration'] + pd.Timedelta(minutes=1)
mask_oneMinTooFew = Phil[mask]
mask_oneMinTooFew
# duration just 1 minute too few

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
4737,365518277,32,2021-01-06 19:39:00,2021-01-06 20:12:00,3214,3214,14503,30,Round Trip,Indego30,standard,2021-01-06,2,19,2021-01-06,2,20,"39.978909,-75.157799","39.978909,-75.157799",0 days 00:32:00
7661,365653263,22,2021-01-09 18:15:00,2021-01-09 18:38:00,3100,3213,11027,30,One Way,Indego30,standard,2021-01-09,5,18,2021-01-09,5,18,"39.927769,-75.151031","39.938869,-75.166634",0 days 00:22:00
11788,365873066,28,2021-01-13 18:18:00,2021-01-13 18:47:00,3214,3214,11746,30,Round Trip,Indego30,standard,2021-01-13,2,18,2021-01-13,2,18,"39.978909,-75.157799","39.978909,-75.157799",0 days 00:28:00
14364,366013270,27,2021-01-15 21:06:00,2021-01-15 21:34:00,3052,3214,18263,30,One Way,Indego30,electric,2021-01-15,4,21,2021-01-15,4,21,"39.947319,-75.156952","39.978909,-75.157799",0 days 00:27:00
33359,367386528,6,2021-02-09 11:38:00,2021-02-09 11:45:00,3012,3213,5129,30,One Way,Indego30,standard,2021-02-09,1,11,2021-02-09,1,11,"39.942181,-75.177467","39.938869,-75.166634",0 days 00:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781174,579419458,13,2022-12-31 15:02:00,2022-12-31 15:16:00,3063,3100,23359,30,One Way,Indego30,electric,2022-12-31,5,15,2022-12-31,5,15,"39.946331,-75.1698","39.927769,-75.151031",0 days 00:13:00
1781328,579438751,8,2022-12-31 17:32:00,2022-12-31 17:41:00,3190,3033,22385,30,One Way,Indego30,electric,2022-12-31,5,17,2022-12-31,5,17,"39.948921,-75.169907","39.95005,-75.156723",0 days 00:08:00
1781343,579439049,8,2022-12-31 17:50:00,2022-12-31 17:59:00,3010,3010,25102,30,Round Trip,Indego30,electric,2022-12-31,5,17,2022-12-31,5,17,"39.947109,-75.166183","39.947109,-75.166183",0 days 00:08:00
1781351,579447376,4,2022-12-31 18:02:00,2022-12-31 18:07:00,3236,3116,22415,365,One Way,Indego365,electric,2022-12-31,5,18,2022-12-31,5,18,"39.96566,-75.17765","39.96006,-75.171982",0 days 00:04:00


In [16]:
mask = calculated_duration + pd.Timedelta(minutes=1) == Phil['givenDuration']
mask_oneMinTooMany = Phil[mask]
mask_oneMinTooMany
# duration just 1 minute too many

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
171,365200960,59,2021-01-01 12:24:00,2021-01-01 13:22:00,3201,3101,14596,30,One Way,Indego30,standard,2021-01-01,4,12,2021-01-01,4,13,"39.955231,-75.166199","39.942951,-75.159554",0 days 00:59:00
215,365201647,45,2021-01-01 13:15:00,2021-01-01 13:59:00,3212,3212,14630,30,Round Trip,Indego30,standard,2021-01-01,4,13,2021-01-01,4,13,"39.963791,-75.18158","39.963791,-75.18158",0 days 00:45:00
222,365201631,26,2021-01-01 13:34:00,2021-01-01 13:59:00,3166,3182,14519,365,One Way,Indego365,standard,2021-01-01,4,13,2021-01-01,4,13,"39.972351,-75.134453","39.950809,-75.169533",0 days 00:26:00
384,365247227,6,2021-01-02 06:18:00,2021-01-02 06:23:00,3169,3033,17642,365,One Way,Indego365,electric,2021-01-02,5,6,2021-01-02,5,6,"39.953819,-75.142632","39.95005,-75.156723",0 days 00:06:00
415,365249927,11,2021-01-02 08:19:00,2021-01-02 08:29:00,3121,3037,5376,365,One Way,Indego365,standard,2021-01-02,5,8,2021-01-02,5,8,"39.97414,-75.180222","39.954239,-75.161377",0 days 00:11:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780884,579374080,29,2022-12-31 10:26:00,2022-12-31 10:54:00,3296,3005,5316,1,One Way,Day Pass,standard,2022-12-31,5,10,2022-12-31,5,10,"39.95134,-75.16758","39.94733,-75.144028",0 days 00:29:00
1780897,579373857,9,2022-12-31 10:33:00,2022-12-31 10:41:00,3294,3101,5131,30,One Way,Indego30,standard,2022-12-31,5,10,2022-12-31,5,10,"39.95174,-75.170631","39.942951,-75.159554",0 days 00:09:00
1780927,579382461,14,2022-12-31 11:02:00,2022-12-31 11:15:00,3033,3032,5337,30,One Way,Indego30,standard,2022-12-31,5,11,2022-12-31,5,11,"39.95005,-75.156723","39.945271,-75.17971",0 days 00:14:00
1781367,579447359,8,2022-12-31 18:29:00,2022-12-31 18:36:00,3101,3010,11004,365,One Way,Indego365,standard,2022-12-31,5,18,2022-12-31,5,18,"39.942951,-75.159554","39.947109,-75.166183",0 days 00:08:00


**Let's assume these are deviations due to rounding in the seconds.
  We now look at those where the duration is significantly violated**

In [17]:
mask = calculated_duration > Phil['givenDuration'] + pd.Timedelta(minutes=1)
mask_DurationUnder = Phil[mask]
mask_DurationUnder
# duration rudimentarily not correct, under the calculated

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
3508,365545144,1440,2021-01-05 16:49:00,2021-01-07 00:08:00,3055,3041,19793,30,One Way,Indego30,electric,2021-01-05,1,16,2021-01-07,3,0,"39.951118,-75.154572","39.968491,-75.13546",1 days
32056,367341249,1440,2021-02-06 21:38:00,2021-02-07 21:45:00,3021,3021,5303,1,Round Trip,Day Pass,standard,2021-02-06,5,21,2021-02-07,6,21,"39.953899,-75.169022","39.953899,-75.169022",1 days
34793,367725064,1440,2021-02-11 09:18:00,2021-02-15 08:57:00,3213,3213,3280,365,Round Trip,Indego365,standard,2021-02-11,3,9,2021-02-15,0,8,"39.938869,-75.166634","39.938869,-75.166634",1 days
36738,367722628,1440,2021-02-14 15:45:00,2021-02-15 16:10:00,3154,3154,3612,30,Round Trip,Indego30,standard,2021-02-14,6,15,2021-02-15,0,16,"39.95924,-75.158211","39.95924,-75.158211",1 days
37195,367828978,1440,2021-02-15 11:08:00,2021-02-17 08:49:00,3073,3073,3703,30,Round Trip,Indego30,standard,2021-02-15,0,11,2021-02-17,2,8,"39.96143,-75.15242","39.96143,-75.15242",1 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774669,578068516,1440,2022-12-23 22:49:00,2022-12-25 04:25:00,3210,3210,16962,30,Round Trip,Indego30,electric,2022-12-23,4,22,2022-12-25,6,4,"39.984921,-75.156677","39.984921,-75.156677",1 days
1779800,579793526,1440,2022-12-30 14:19:00,2023-01-02 04:25:00,3212,3212,23229,1,Round Trip,Day Pass,electric,2022-12-30,4,14,2023-01-02,0,4,"39.963791,-75.18158","39.963791,-75.18158",1 days
1780277,579859025,1440,2022-12-30 17:26:00,2023-01-02 10:42:00,3071,3071,11765,30,Round Trip,Indego30,standard,2022-12-30,4,17,2023-01-02,0,10,"39.961731,-75.187881","39.961731,-75.187881",1 days
1781193,579647620,1440,2022-12-31 15:15:00,2023-01-01 15:46:00,3004,3057,11948,1,One Way,Day Pass,standard,2022-12-31,5,15,2023-01-01,6,15,"39.953781,-75.163742","39.96439,-75.179871",1 days


In [18]:
mask = mask_DurationUnder['givenDuration'] >= '1 days'
mask_DurationOver1440 = mask_DurationUnder[mask]
mask_DurationOver1440

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
3508,365545144,1440,2021-01-05 16:49:00,2021-01-07 00:08:00,3055,3041,19793,30,One Way,Indego30,electric,2021-01-05,1,16,2021-01-07,3,0,"39.951118,-75.154572","39.968491,-75.13546",1 days
32056,367341249,1440,2021-02-06 21:38:00,2021-02-07 21:45:00,3021,3021,5303,1,Round Trip,Day Pass,standard,2021-02-06,5,21,2021-02-07,6,21,"39.953899,-75.169022","39.953899,-75.169022",1 days
34793,367725064,1440,2021-02-11 09:18:00,2021-02-15 08:57:00,3213,3213,3280,365,Round Trip,Indego365,standard,2021-02-11,3,9,2021-02-15,0,8,"39.938869,-75.166634","39.938869,-75.166634",1 days
36738,367722628,1440,2021-02-14 15:45:00,2021-02-15 16:10:00,3154,3154,3612,30,Round Trip,Indego30,standard,2021-02-14,6,15,2021-02-15,0,16,"39.95924,-75.158211","39.95924,-75.158211",1 days
37195,367828978,1440,2021-02-15 11:08:00,2021-02-17 08:49:00,3073,3073,3703,30,Round Trip,Indego30,standard,2021-02-15,0,11,2021-02-17,2,8,"39.96143,-75.15242","39.96143,-75.15242",1 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774669,578068516,1440,2022-12-23 22:49:00,2022-12-25 04:25:00,3210,3210,16962,30,Round Trip,Indego30,electric,2022-12-23,4,22,2022-12-25,6,4,"39.984921,-75.156677","39.984921,-75.156677",1 days
1779800,579793526,1440,2022-12-30 14:19:00,2023-01-02 04:25:00,3212,3212,23229,1,Round Trip,Day Pass,electric,2022-12-30,4,14,2023-01-02,0,4,"39.963791,-75.18158","39.963791,-75.18158",1 days
1780277,579859025,1440,2022-12-30 17:26:00,2023-01-02 10:42:00,3071,3071,11765,30,Round Trip,Indego30,standard,2022-12-30,4,17,2023-01-02,0,10,"39.961731,-75.187881","39.961731,-75.187881",1 days
1781193,579647620,1440,2022-12-31 15:15:00,2023-01-01 15:46:00,3004,3057,11948,1,One Way,Day Pass,standard,2022-12-31,5,15,2023-01-01,6,15,"39.953781,-75.163742","39.96439,-75.179871",1 days


**These are trips with over 1440min that can be deleted**

In [19]:
reversed_mask = ~ mask
mask_DurationUnderCalculated = mask_DurationUnder[reversed_mask]
mask_DurationUnderCalculated

# these are trips where the duration is incorrect. it should be bigger 

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
63111,370449593,728,2021-03-13 15:50:00,2021-03-14 04:58:00,3182,3182,11910,1,Round Trip,Day Pass,standard,2021-03-13,5,15,2021-03-14,6,4,"39.950809,-75.169533","39.950809,-75.169533",0 days 12:08:00
63271,370449410,670,2021-03-13 16:49:00,2021-03-14 04:59:00,3016,3039,3275,1,One Way,Day Pass,standard,2021-03-13,5,16,2021-03-14,6,4,"39.968922,-75.154701","39.97121,-75.159698",0 days 11:10:00
63707,370495976,675,2021-03-13 21:57:00,2021-03-14 10:12:00,3159,3159,18160,1,Round Trip,Day Pass,electric,2021-03-13,5,21,2021-03-14,6,10,"39.95121,-75.199623","39.95121,-75.199623",0 days 11:15:00
63798,370440738,60,2021-03-14 01:52:00,2021-03-14 03:52:00,3005,3005,3559,1,Round Trip,Day Pass,standard,2021-03-14,6,1,2021-03-14,6,3,"39.94733,-75.144028","39.94733,-75.144028",0 days 01:00:00
63799,370439929,19,2021-03-14 01:53:00,2021-03-14 03:12:00,3075,3050,17646,30,One Way,Indego30,electric,2021-03-14,6,1,2021-03-14,6,3,"39.967178,-75.161247","39.953388,-75.154259",0 days 00:19:00
63800,370440029,23,2021-03-14 01:53:00,2021-03-14 03:16:00,3150,3035,14650,30,One Way,Indego30,standard,2021-03-14,6,1,2021-03-14,6,3,"39.928841,-75.170212","39.962711,-75.194191",0 days 00:23:00
63801,370440735,57,2021-03-14 01:55:00,2021-03-14 03:52:00,3005,3005,11943,1,Round Trip,Day Pass,standard,2021-03-14,6,1,2021-03-14,6,3,"39.94733,-75.144028","39.94733,-75.144028",0 days 00:57:00
63802,370439928,19,2021-03-14 01:56:00,2021-03-14 03:15:00,3055,3041,11870,30,One Way,Indego30,standard,2021-03-14,6,1,2021-03-14,6,3,"39.951118,-75.154572","39.968491,-75.13546",0 days 00:19:00
63803,370463647,198,2021-03-14 01:57:00,2021-03-14 06:15:00,3050,3037,5372,1,One Way,Day Pass,standard,2021-03-14,6,1,2021-03-14,6,6,"39.953388,-75.154259","39.954239,-75.161377",0 days 03:18:00
1002469,464622855,311,2022-03-12 22:46:00,2022-03-13 04:57:00,3187,3059,18438,30,One Way,Indego30,electric,2022-03-12,5,22,2022-03-13,6,4,"39.957249,-75.172318","39.96244,-75.161209",0 days 05:11:00


**For these ten trips we have to correct the duration**

In [20]:
mask = calculated_duration + pd.Timedelta(minutes=1) < Phil['givenDuration']
mask_DurationOverCalculated = Phil[mask]
mask_DurationOverCalculated
# duration rudimentarily not correct, over the calculated


,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
813678,441773921,735,2021-11-06 15:44:00,2021-11-07 02:59:00,3059,3041,3453,1,One Way,Day Pass,standard,2021-11-06,5,15,2021-11-07,6,2,"39.96244,-75.161209","39.968491,-75.13546",0 days 12:15:00
814488,441773055,433,2021-11-06 20:42:00,2021-11-07 02:55:00,3202,3108,19257,30,One Way,Indego30,electric,2021-11-06,5,20,2021-11-07,6,2,"39.95134,-75.16758","39.953159,-75.165512",0 days 07:13:00
814519,441773024,411,2021-11-06 21:04:00,2021-11-07 02:55:00,3108,3108,3597,30,Round Trip,Indego30,standard,2021-11-06,5,21,2021-11-07,6,2,"39.953159,-75.165512","39.953159,-75.165512",0 days 06:51:00
814679,441913222,1049,2021-11-06 23:20:00,2021-11-07 15:49:00,3255,3255,3320,30,Round Trip,Indego30,standard,2021-11-06,5,23,2021-11-07,6,15,"39.950951,-75.164383","39.950951,-75.164383",0 days 17:29:00
814779,441772650,179,2021-11-07 00:56:00,2021-11-07 02:55:00,3108,3108,11831,30,Round Trip,Indego30,standard,2021-11-07,6,0,2021-11-07,6,2,"39.953159,-75.165512","39.953159,-75.165512",0 days 02:59:00
814842,441761482,40,2021-11-07 01:48:00,2021-11-07 01:28:00,3098,3185,11938,30,One Way,Indego30,standard,2021-11-07,6,1,2021-11-07,6,1,"39.934311,-75.160423","39.951691,-75.158882",0 days 00:40:00
814843,441760975,12,2021-11-07 01:49:00,2021-11-07 01:01:00,3235,3245,11816,30,One Way,Indego30,standard,2021-11-07,6,1,2021-11-07,6,1,"39.959999,-75.1651","39.97887,-75.157768",0 days 00:12:00
1682797,558054442,1409,2022-11-05 14:01:00,2022-11-06 12:30:00,3058,3051,19158,1,One Way,Day Pass,electric,2022-11-05,5,14,2022-11-06,6,12,"39.967159,-75.170013","39.967442,-75.175072",0 days 23:29:00
1683260,557866179,829,2022-11-05 15:32:00,2022-11-06 04:21:00,3015,3015,11747,1,Round Trip,Day Pass,standard,2022-11-05,5,15,2022-11-06,6,4,"39.94735,-75.148857","39.94735,-75.148857",0 days 13:49:00
1683811,557865588,714,2022-11-05 17:27:00,2022-11-06 04:21:00,3117,3041,23054,30,One Way,Indego30,electric,2022-11-05,5,17,2022-11-06,6,4,"39.978409,-75.223991","39.968491,-75.13546",0 days 11:54:00


**For these 15 trips we have to correct the duration**

**index --> 441761482, 441760975, 557784542, 557784541, 557784539, 557784538, 557784535, 557784534, 557784533**
**There are 9 trips where start_time is bigger then end_time. We can delete these rows**

In [21]:
Phil = Phil[Phil['start_time'] <= Phil['end_time']]
# now it includes only trips where start_time is smaller then end_time

**Here we see if the same data frame where 1440 minutes was given and the actual duration is over 1440 minutes is the same**

In [22]:
mask_DurationOver1440 = mask_DurationOver1440.drop('givenDuration', axis=1)
r = mask_DurationOver1440.shape == mask1440.shape and (mask_DurationOver1440 == mask1440).all().all()
r

True

In [23]:
Phil.drop(Phil[Phil["duration"]==1440].index, inplace = True)
Phil

C:\Users\r-cet\AppData\Local\Temp\ipykernel_6292\2986160019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Phil.drop(Phil[Phil["duration"]==1440].index, inplace = True)


,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,givenDuration
0,365184634,15,2021-01-01 00:05:00,2021-01-01 00:20:00,3125,3157,14542,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.943909,-75.167351","39.925449,-75.159538",0 days 00:15:00
1,365184633,14,2021-01-01 00:06:00,2021-01-01 00:20:00,3022,3010,5294,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.95472,-75.183228","39.947109,-75.166183",0 days 00:14:00
2,365184632,33,2021-01-01 00:07:00,2021-01-01 00:40:00,3066,3201,14586,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.94561,-75.173477","39.955231,-75.166199",0 days 00:33:00
3,365184631,9,2021-01-01 00:13:00,2021-01-01 00:22:00,3045,3034,14683,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947922,-75.162369","39.933151,-75.162483",0 days 00:09:00
4,365184630,13,2021-01-01 00:37:00,2021-01-01 00:50:00,3052,3040,3330,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947319,-75.156952","39.962891,-75.166061",0 days 00:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781469,579499101,23,2022-12-31 23:44:00,2023-01-01 00:07:00,3244,3287,14674,30,One Way,Indego30,standard,2022-12-31,5,23,2023-01-01,6,0,"39.938648,-75.16674","39.943668,-75.216057",0 days 00:23:00
1781470,579499099,17,2022-12-31 23:45:00,2023-01-01 00:02:00,3295,3054,23113,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950279,-75.160271","39.962502,-75.174202",0 days 00:17:00
1781471,579499097,8,2022-12-31 23:51:00,2022-12-31 23:59:00,3255,3046,16897,30,One Way,Indego30,electric,2022-12-31,5,23,2022-12-31,5,23,"39.950951,-75.164383","39.950119,-75.144722",0 days 00:08:00
1781472,579499095,23,2022-12-31 23:54:00,2023-01-01 00:16:00,3255,3113,22646,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950951,-75.164383","39.97472,-75.197807",0 days 00:23:00


**Here we correct the duration for mask_DurationUnderCalculation and mask_DurationOverCalculated****

In [24]:
duration_minutes = (Phil['end_time'] - Phil['start_time']).dt.total_seconds() / 60

In [25]:
trip_ids = [370449593, 370449410, 370495976, 370440738, 370439929, 370440029, 370440735, 370439928, 370463647, 464622855]
mask1 = Phil['trip_id'].isin(trip_ids)
Phil.loc[mask1, 'duration'] = duration_minutes

In [26]:
trip_ids = [441773921, 441773055, 441773024, 441913222, 441772650, 558054442, 557866179, 557865588, 557865552, 557865199,
           557865148, 557786020, 557812231, 557784875, 557784871]
mask2 = Phil['trip_id'].isin(trip_ids)
Phil.loc[mask2, 'duration'] = duration_minutes

In [27]:
Phil = Phil.drop('givenDuration', axis=1)
Phil

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
0,365184634,15,2021-01-01 00:05:00,2021-01-01 00:20:00,3125,3157,14542,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.943909,-75.167351","39.925449,-75.159538"
1,365184633,14,2021-01-01 00:06:00,2021-01-01 00:20:00,3022,3010,5294,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.95472,-75.183228","39.947109,-75.166183"
2,365184632,33,2021-01-01 00:07:00,2021-01-01 00:40:00,3066,3201,14586,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.94561,-75.173477","39.955231,-75.166199"
3,365184631,9,2021-01-01 00:13:00,2021-01-01 00:22:00,3045,3034,14683,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947922,-75.162369","39.933151,-75.162483"
4,365184630,13,2021-01-01 00:37:00,2021-01-01 00:50:00,3052,3040,3330,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947319,-75.156952","39.962891,-75.166061"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781469,579499101,23,2022-12-31 23:44:00,2023-01-01 00:07:00,3244,3287,14674,30,One Way,Indego30,standard,2022-12-31,5,23,2023-01-01,6,0,"39.938648,-75.16674","39.943668,-75.216057"
1781470,579499099,17,2022-12-31 23:45:00,2023-01-01 00:02:00,3295,3054,23113,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950279,-75.160271","39.962502,-75.174202"
1781471,579499097,8,2022-12-31 23:51:00,2022-12-31 23:59:00,3255,3046,16897,30,One Way,Indego30,electric,2022-12-31,5,23,2022-12-31,5,23,"39.950951,-75.164383","39.950119,-75.144722"
1781472,579499095,23,2022-12-31 23:54:00,2023-01-01 00:16:00,3255,3113,22646,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950951,-75.164383","39.97472,-75.197807"


**Here we check if Date_s is bigger then Date_e**

In [28]:
Phil['Date_s'] = pd.to_datetime(Phil['Date_s'])
Phil['Date_e'] = pd.to_datetime(Phil['Date_e'])
mask = Phil['Date_e'] - Phil['Date_s'] > pd.Timedelta(days=1)
mask = Phil[mask]
mask

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates


In [29]:
Phil

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
0,365184634,15,2021-01-01 00:05:00,2021-01-01 00:20:00,3125,3157,14542,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.943909,-75.167351","39.925449,-75.159538"
1,365184633,14,2021-01-01 00:06:00,2021-01-01 00:20:00,3022,3010,5294,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.95472,-75.183228","39.947109,-75.166183"
2,365184632,33,2021-01-01 00:07:00,2021-01-01 00:40:00,3066,3201,14586,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.94561,-75.173477","39.955231,-75.166199"
3,365184631,9,2021-01-01 00:13:00,2021-01-01 00:22:00,3045,3034,14683,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947922,-75.162369","39.933151,-75.162483"
4,365184630,13,2021-01-01 00:37:00,2021-01-01 00:50:00,3052,3040,3330,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947319,-75.156952","39.962891,-75.166061"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781469,579499101,23,2022-12-31 23:44:00,2023-01-01 00:07:00,3244,3287,14674,30,One Way,Indego30,standard,2022-12-31,5,23,2023-01-01,6,0,"39.938648,-75.16674","39.943668,-75.216057"
1781470,579499099,17,2022-12-31 23:45:00,2023-01-01 00:02:00,3295,3054,23113,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950279,-75.160271","39.962502,-75.174202"
1781471,579499097,8,2022-12-31 23:51:00,2022-12-31 23:59:00,3255,3046,16897,30,One Way,Indego30,electric,2022-12-31,5,23,2022-12-31,5,23,"39.950951,-75.164383","39.950119,-75.144722"
1781472,579499095,23,2022-12-31 23:54:00,2023-01-01 00:16:00,3255,3113,22646,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950951,-75.164383","39.97472,-75.197807"


In [30]:
Phil.to_csv('cleaned_Phil.csv', index=False)